In [1]:
import xgboost
import joblib
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from quadratic_weighted_kappa import quadratic_weighted_kappa

In [2]:
x = joblib.load('essay_ngram_asap7')
y = joblib.load("score_asap7")

In [3]:
x.shape

(1569, 14974)

In [4]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
print(kf)

KFold(n_splits=5, random_state=42, shuffle=True)


In [11]:
model2 = xgboost.XGBRegressor(objective ='reg:squarederror',
                colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=1000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

In [12]:
qwk_scores = []
test_indices = []
pred_labels = []


for train_index, test_index in kf.split(x):
    
    X_train, X_test, Y_train, Y_test = x[train_index], x[test_index], y[train_index], y[test_index]
         
    model2.fit(X_train, Y_train)
    
    predict = model2.predict(X_test)
    predict = np.round(predict, 3)
    
    pred_labels.extend(predict)
    test_indices.extend(test_index)

    result = quadratic_weighted_kappa(Y_test, predict)
    
    print("Qwk : ", result)
    
    qwk_scores.append(result)
    
np.mean(qwk_scores)

Qwk :  0.7209694674621876
Qwk :  0.7016612383020155
Qwk :  0.725108817145349
Qwk :  0.7060060134330626
Qwk :  0.6811747917130737


0.7069840656111377

In [ ]:
pred_labels_int = list(map(int, pred_labels))

In [13]:
#new_score = np.zeros(1569, dtype=np.int64)
new_score = np.zeros(1569)

In [14]:
#new_score[test_indices] = pred_labels_int
new_score[test_indices] = pred_labels

In [20]:
joblib.dump(new_score, 'score_model_gb_normalized_ngram')

['score_model_rr_normalized_ngram']

In [15]:
quadratic_weighted_kappa(new_score, y)

0.707564433444519

In [16]:
# Save the original score in floating number
joblib.dump(new_score, 'score_model_gb_normalized_float_ngram')

['score_model_gb_normalized_float_ngram']

In [ ]:
new_score